# Weights & Biases using Keras for MNIST Dataset

In [22]:
# For Weights and Biases
!pip install -qq wandb
# To download the dataset
!pip install python-mnist

In [23]:
## General Dependencies
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

# For Deep Learning
import tensorflow as tf
print("TF: ", tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models

# For MLOps
import wandb
print("W&B: ", wandb.__version__)
from wandb.keras import WandbCallback

# For medMNIST dataset
#print("medMNIST: ", mnist.__version__)
#INFO = 
# import medmnist
# print("medMNIST: ", medmnist.__version__)
# from medmnist import INFO

from keras.datasets import mnist
# import mnist
# print("medMNIST: ", medmnist.__version__)
# from mnist import

TF:  2.8.0
W&B:  0.12.16


In [24]:
#Authorizing the users

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [25]:
#Config to run the file

configs = dict(
    data_flag = 'mnist',
    image_width = 32,
    image_height = 32,
    batch_size = 128,
    model_name = 'vgg16',
    pretrain_weights = 'imagenet',
    epochs = 20,
    init_learning_rate = 0.001,
    lr_decay_rate = 0.1,
    optimizer = 'adam',
    loss_fn = 'sparse_categorical_crossentropy',
    metrics = ['acc'],
    earlystopping_patience = 5
)

In [26]:
#info = INFO[configs['data_flag']]
configs['class_names'] = {0: 0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9}
#configs['class_names'] = info['label']
# configs['image_channels'] = info['n_channels']
configs['image_channels'] = 1

print(configs['class_names'])
print(configs['image_channels'])
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_labels)
print(test_labels)
#info

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}
1
[5 0 4 ... 5 6 8]
[7 2 1 ... 4 5 6]


In [27]:
# Preparing Dataset
def download_and_prepare_dataset(data_info: dict):
    """
    Utility function to download the dataset and return train/valid/test images/labels.

    Arguments:
        data_info (dict): Dataset metadata
    """
    data_path = tf.keras.utils.get_file(origin=data_info['url'], md5_hash=data_info['MD5'])

    with np.load(data_path) as data:
        # Get images
        train_images = data['train_images']
        valid_images = data['val_images']
        test_images = data['test_images']

        # Get labels
        train_labels = data['train_labels'].flatten()
        valid_labels = data['val_labels'].flatten()
        test_labels = data['test_labels'].flatten()

    return train_images, train_labels, valid_images, valid_labels, test_images, test_labels

In [28]:
# For demonstration purposes
log_full = False #@param {type:"boolean"}

if log_full:
    log_train_samples = len(train_images)
else:
    log_train_samples = 1000 

print(f'Number of train images : {log_train_samples} to be logged')

Number of train images : 1000 to be logged


In [29]:
%%time

# Initialize a new W&B run
run = wandb.init(project='mnist', group='viz_data')

# Intialize a W&B Artifacts
ds = wandb.Artifact("mnist_dataset", "dataset")

# Initialize an empty table
train_table = wandb.Table(columns=[], data=[])
# Add training data
train_table.add_column('image', train_images[:log_train_samples])
# Add training label_id
train_table.add_column('label_id', train_labels[:log_train_samples])
# Add training class names
train_table.add_computed_columns(lambda ndx, row:{
    "images": wandb.Image(row["image"]),
    "class_names": configs['class_names'][row["label_id"]]
    })

# Add the table to the Artifact
ds['train_data'] = train_table

# Let's do the same for the validation data
test_table = wandb.Table(columns=[], data=[])
test_table.add_column('image', test_images)
test_table.add_column('label_id', test_labels)
test_table.add_computed_columns(lambda ndx, row:{
    "images": wandb.Image(row["image"]),
    "class_name": configs['class_names'][row["label_id"]]
    })
ds['test_data'] = test_table

# Save the dataset as an Artifact
ds.save()

# Finish the run
wandb.finish()

CPU times: user 9.11 s, sys: 1.75 s, total: 10.9 s
Wall time: 30.6 s


In [30]:
#@title
@tf.function
def preprocess(image: tf.Tensor, label: tf.Tensor):
    """
    Preprocess the image tensors and parse the labels
    """
    # Preprocess images
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Parse label
    label = tf.cast(label, tf.float32)
    
    return image, label


def prepare_dataloader(images: np.ndarray,
                       labels: np.ndarray,
                       loader_type: str='train',
                       batch_size: int=128):
    """
    Utility function to prepare dataloader.
    """
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))

    if loader_type=='train':
        dataset = dataset.shuffle(1024)

    dataloader = (
        dataset
        .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    return dataloader

In [31]:
trainloader = prepare_dataloader(train_images, train_labels, 'train', configs.get('batch_size', 64))
#validloader = prepare_dataloader(valid_images, valid_labels, 'valid', configs.get('batch_size', 64))
testloader = prepare_dataloader(test_images, test_labels, 'test', configs.get('batch_size', 64))

In [32]:
img_augmentation = models.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomFlip()],
    name="img_augmentation",
)

# Model


In [33]:
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

def get_model():
  
    # inputs = layers.Input(input_shape)
    # resize_img = layers.Resizing(resize[0], resize[1], interpolation='bilinear')(inputs)
    # augment_img = img_augmentation(resize_img)
  
    # base_model = tf.keras.applications.VGG16(include_top=False, 
    #                                          weights=configs['pretrain_weights'], 
    #                                          input_shape=resize)
    #                                          #input_tensor=augment_img)
    # base_model.trainabe = True

    
    # x = base_model.output
    # x = layers.GlobalAveragePooling2D()(x)
    # x = layers.Dropout(dropout_rate)(x)
    # outputs = layers.Dense(num_classes, activation=output_activation)(x)

    # return models.Model(inputs, outputs)
    input = Input(shape=(28, 28, 1))
    x = Conv2D(4, (5, 5), strides=1, activation="relu", padding="same")(input)
    x = Conv2D(8, (5, 5), strides=2, activation="relu", padding="same")(x)
    x = Conv2D(12, (4, 4), strides=2, activation="relu", padding="same")(x)
    x = Flatten()(x)
    x = Dense(200, activation="relu")(x)
    x = Dropout(0.1)(x)
    output = Dense(10, activation="softmax")(x)
    model = Model(inputs=input, outputs=output)
    model.compile(loss="categorical_crossentropy",
                  optimizer=Adam(),
                  metrics=["accuracy"])
    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 4)         104       
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 8)         808       
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 12)          1548      
                                                                 
 flatten (Flatten)           (None, 588)               0         
                                                                 
 dense (Dense)               (None, 200)               117800    
                                                                 
 dropout (Dropout)           (None, 200)               0     

In [34]:
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=configs['earlystopping_patience'], verbose=0, mode='auto',
    restore_best_weights=True
)

In [35]:
def lr_scheduler(epoch, lr):
    # log the current learning rate onto W&B
    if wandb.run is None:
        raise wandb.Error("You must call wandb.init() before WandbCallback()")

    wandb.log({'learning_rate': lr}, commit=False)
    
    if epoch < 7:
        return lr
    else:
        return lr * tf.math.exp(-configs['lr_decay_rate'])

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [36]:
def train(config: dict, 
          callbacks: list,
          verbose: int=0):
    """
    Utility function to train the model.

    Arguments:
        config (dict): Dictionary of hyperparameters.
        callbacks (list): List of callbacks passed to `model.fit`.
        verbose (int): 0 for silent and 1 for progress bar.
    """

    # Initalize model
    tf.keras.backend.clear_session()
    #resize=(config.image_width, config.image_height, config.image_channels)
    model = get_model()

    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=config.init_learning_rate)
    model.compile(opt,
                  config.loss_fn,
                  metrics=config.metrics)

    # Train the model
    _ = model.fit(trainloader,
                  epochs=config.epochs,
                  validation_data=testloader,
                  callbacks=callbacks,
                  verbose=verbose)

    return model

In [37]:
# Initialize the W&B run
run = wandb.init(project='mnist', config=configs, job_type='train')
config = wandb.config

# Define WandbCallback for experiment tracking
wandb_callback = WandbCallback(monitor='val_loss',
                               log_weights=True,
                               log_evaluation=True,
                               validation_steps=5)

# callbacks
callbacks = [earlystopper, wandb_callback, lr_callback]

# Train
model = train(config, callbacks=callbacks, verbose=1)

# Evaluate the trained model
loss, acc = model.evaluate(testloader)
wandb.log({'evaluate/accuracy': acc})

# Close the W&B run.
wandb.finish()

Epoch 1/20
469/469 [==============================] - 47s 99ms/step - loss: 0.3058 - acc: 0.9086 - val_loss: 0.0915 - val_acc: 0.9700 - _timestamp: 1651974397.0000 - _runtime: 53.0000 - lr: 0.0010
Epoch 2/20
469/469 [==============================] - 46s 98ms/step - loss: 0.0938 - acc: 0.9711 - val_loss: 0.0585 - val_acc: 0.9804 - _timestamp: 1651974478.0000 - _runtime: 134.0000 - lr: 0.0010
Epoch 3/20
469/469 [==============================] - 46s 98ms/step - loss: 0.0673 - acc: 0.9787 - val_loss: 0.0511 - val_acc: 0.9833 - _timestamp: 1651974560.0000 - _runtime: 216.0000 - lr: 0.0010
Epoch 4/20
469/469 [==============================] - 46s 98ms/step - loss: 0.0507 - acc: 0.9846 - val_loss: 0.0519 - val_acc: 0.9829 - _timestamp: 1651974642.0000 - _runtime: 298.0000 - lr: 0.0010
Epoch 5/20
469/469 [==============================] - 45s 96ms/step - loss: 0.0426 - acc: 0.9864 - val_loss: 0.0445 - val_acc: 0.9859 - _timestamp: 1651974723.0000 - _runtime: 379.0000 - lr: 0.0010
Epoch 6/20


acc,▁▆▇▇▇▇█████
epoch,▁▂▂▃▄▅▅▆▇▇█
evaluate/accuracy,▁
learning_rate,████████▅▃▁
loss,█▃▂▂▂▂▁▁▁▁▁
val_acc,▁▅▆▆▇▇▇▇▇▇█
val_loss,█▄▃▃▂▁▁▁▂▂▁
acc,0.99512
best_epoch,5
best_val_loss,0.03926
epoch,10


In [38]:
wandb.init()
predictions = model(test_images)
ground_truth = test_labels
wandb.log({"roc" : wandb.plot.roc_curve( ground_truth, predictions,
                        labels=[0,1,2,3,4,5,6,7,8,9], classes_to_plot=None)})